In [168]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import warnings,os,sys,datetime,time

import pandas as pd

from scipy.linalg import block_diag

In [264]:
sizeI = 10
sizeH = 250
sizeO = 25

N = sizeI + sizeH + sizeO

Wt = rnd.randn(N,N)
W = (Wt + Wt.T)/2
for k in range(W.shape[0]): W[k,k] = 0
b = rnd.randn(N)

alpha = lambda x: np.dot(W,x) + b
softmax = lambda x,T=1: T/(1+np.exp(-x))
sigma = lambda x: softmax(alpha(x))

In [265]:
TI = np.concatenate([np.eye(sizeI),np.zeros([sizeI,sizeH+sizeO])],axis=1)
TH = np.concatenate([np.zeros([sizeH,sizeI]),np.eye(sizeH),np.zeros([sizeH,sizeO])],axis=1)
TO = np.concatenate([np.zeros([sizeO,sizeI+sizeH]),np.eye(sizeO)],axis=1)

In [266]:
def settle_nw(s,eps_min=0.001,max_steps=10000,verbose=False):
    loss = []
    states = []
    eps = np.inf
    ctr = 0
    while eps>eps_min:
        states.append(s)
        sn = sigma(s)
        eps = np.abs(np.sum(np.dot(TH,states[-1]) - np.dot(TH,sn)))/s.shape[0]
        if verbose: print('Step {0}/{1} --> loss = {2}'.format(ctr,max_steps,eps))
        loss.append(eps)
        s = np.concatenate([np.dot(TI,states[-1]),np.dot(TH,sn),np.dot(TO,sn)],axis=0)
        ctr += 1
        if ctr>max_steps: break
    terminated = True
    if ctr>max_steps: terminated = False
    return s,terminated,loss,states

In [267]:
fixedInput = rnd.randint(0,2,sizeI)

In [268]:
s0 = rnd.randn(N)
s1 = rnd.randn(N)

s0[:sizeI] = fixedInput
s1[:sizeI] = fixedInput

sinf0,terminated0,loss0,states0 = settle_nw(s0)
sinf1,terminated1,loss1,states1 = settle_nw(s1)

out0 = np.dot(TO,sinf0)
out1 = np.dot(TO,sinf1)

print('Terminated>{0}  --  Like inputs yield correlation {1:.3f}'.format(terminated0 and terminated1,np.corrcoef(out0,out1)[0,1]))

Terminated>False  --  Like inputs yield correlation 0.957


In [269]:
s0 = rnd.randn(N)
s1 = rnd.randn(N)

s0[:sizeI] = fixedInput
s1[:sizeI] = np.ones_like(fixedInput) - fixedInput

sinf0,terminated0,loss0,states0 = settle_nw(s0)
sinf1,terminated1,loss1,states1 = settle_nw(s1)

out0 = np.dot(TO,sinf0)
out1 = np.dot(TO,sinf1)

print('Terminated>{0}  --  Unlike inputs yield correlation {1:.3f}'.format(terminated0 and terminated1,np.corrcoef(out0,out1)[0,1]))

Terminated>False  --  Unlike inputs yield correlation 0.810


In [270]:
out0

array([9.99856878e-01, 9.99999647e-01, 9.99999999e-01, 9.18748650e-09,
       5.02092541e-02, 3.38252469e-12, 9.97280878e-01, 2.31013746e-06,
       3.75055489e-02, 1.06540066e-07, 9.99997255e-01, 9.38547146e-01,
       2.31661244e-04, 6.77635375e-05, 9.99999550e-01, 9.99999919e-01,
       9.99994240e-01, 6.48148696e-06, 9.95448819e-01, 5.51502877e-08,
       1.65397296e-03, 9.99843031e-01, 5.13547033e-01, 1.61236326e-07,
       9.99993759e-01])